# Classification
So far the examples used so far have been for linear regression models, however, this notebook will examine the same setup and look at a classification example to see how accurate the method is